In [3]:
import pyccl as ccl
print(f"I am using pyccl version {ccl.__version__}")
import numpy as np
import matplotlib.pyplot as plt
from scripts.presets import Presets
from scripts.data_vectors import DataVectors
from scripts.data_vector_metrics import DataVectorMetrics


I am using pyccl version 3.0.2


In [4]:
# This analysis will be done for a cosmology with the following parameters
# if you want to change the cosmology, you can do it here or just run without it.
# For now, we will keep it as is.
#cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, sigma8=0.83, n_s=0.96)

# The class has default values for the maximal redshift and the resolution of redshifts.
# You can change them if you want to or run the default values.
# Since we are investigating the impact of the resolution of redshifts,
# we will keep the maximal redshift as is (for now) and only change the redshift resolution.
redshift_max = 3.5
redshift_resolution_1 = 300  # this is the first resolution of redshifts
redshift_resolution_2 = 5000  # this is the second resolution of redshifts
forecast_year = "1"
# Please note that the redshift resolution is the number of redshifts between 0 and redshift_max.
# Additionally, a lot of parametric sweeping is already coded into helper functions 
# so we will not need to worry about that for now.

# We will also investigate ell binning but for this particular exercise,
# we will keep the ell binning at default values.
#ell_min = 20
#ell_max = 2000
#n_ell = 50

# We now initialize the class by only passing the redshift information.
presets = Presets(redshift_max=redshift_max,
                  redshift_resolution=redshift_resolution_1,
                  forecast_year=forecast_year)

data = DataVectors(presets)
cl_gc = np.array(data.galaxy_clustering_cls(want_all=True))
cl_ggl = np.array(data.galaxy_galaxy_lensing_cls(want_all=True))
cl_cs = np.array(data.cosmic_shear_cls(want_all=True))


presets_1 = Presets(redshift_max=3.5,
                  redshift_resolution=redshift_resolution_2,
                  forecast_year=forecast_year)

data_1 = DataVectors(presets_1)
cl_gc_1 = np.array(data_1.galaxy_clustering_cls(want_all=True))
cl_ggl_1 = np.array(data_1.galaxy_galaxy_lensing_cls(want_all=True))
cl_cs_1 = np.array(data_1.cosmic_shear_cls(want_all=True))


metric = DataVectorMetrics(presets)

print(metric.get_delta_chi2(cl_gc, cl_ggl, cl_cs,cl_gc_1, cl_ggl_1, cl_cs_1))



Data saved to data_output/angular_power_spectra/galaxy_clustering_correlations_y1.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_cls_y1_zmax3.5_res300_ccl_v3.0.2.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_y1.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_cls_y1_zmax3.5_res300_ccl_v3.0.2.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_correlations_y1.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_cls_y1_zmax3.5_res300_ccl_v3.0.2.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_correlations_y1.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_cls_y1_zmax3.5_res5000_ccl_v3.0.2.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_y1.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_cls_y1_zmax3.5_res5000_ccl_v3.0.2.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_correlations